In [ ]:
from dotenv import load_dotenv
import os
import numpy as np
import pandas as pd
load_dotenv()

SEED = 1303

dfs = []
path_dados = os.getenv("PATH_DADOS")


for path in os.listdir(path_dados):
    if path.startswith('df_com_pred'):
        

        df = pd.read_parquet(f'{path_dados}/{path}')
        dfs.append(df)

dfs = pd.concat(dfs)

dfs.head()


ArrowMemoryError: malloc of size 650117120 failed

In [ ]:
colunas_nescessarias = ['texto','pred_prob']

df_final = dfs[colunas_nescessarias].copy()

In [ ]:
df_final['prob_negativa'] = df_final['pred_prob'].apply(lambda x : x[0] if x[0]>0.6 else None)
df_final['prob_positiva'] = df_final['pred_prob'].apply(lambda x : x[1] if x[1]>0.6 else None)
df_final = df_final.dropna(subset=['prob_negativa','prob_positiva'],how='all')

df_final['classe'] = df_final['pred_prob'].apply(lambda x : np.argmax(x))
df_final




,texto,pred_prob,prob_negativa,prob_positiva,classe
0,EXECUÇÃO FISCAL. AGRAVO DE INSTRUMENTO. INMETR...,"[0.9999940329648943, 5.967035105681065e-06]",0.999994,NaN,0
1,PENAL E PROCESSUAL PENAL. TRÁFICO INTERNACIONA...,"[0.9868643618286582, 0.013135638171341812]",0.986864,NaN,0
2,EMBARGOS DECLARATÓRIOS. CONTRADIÇÃO. ERRO MAT...,"[0.9868643618286582, 0.013135638171341812]",0.986864,NaN,0
3,PENAL. PROCESSUAL PENAL. CRIME CONTRA O SISTEM...,"[0.9868643618286582, 0.013135638171341812]",0.986864,NaN,0
4,PENAL E PROCESSO PENAL. ESTELIONATO PREVIDENCI...,"[0.9868643618286582, 0.013135638171341812]",0.986864,NaN,0
...,...,...,...,...,...
497173,\n EMENTA \n ADMINISTRATIVO. SERVIDORA PÚBLICA...,"[0.990146682919641, 0.009853317080359048]",0.990147,NaN,0
497174,PROCESSO Nº: 0810711-49.2023.4.05.0000 - AGRAV...,"[0.9868643618286582, 0.013135638171341812]",0.986864,NaN,0
497175,PROCESSO Nº: 0800194-96.2023.4.05.8402 - APELA...,"[0.9868643618286582, 0.013135638171341812]",0.986864,NaN,0
497176,\n EMENTA \n \n ADMINISTRATIVO. APELAÇÃO E REM...,"[0.9868643618286582, 0.013135638171341812]",0.986864,NaN,0


In [ ]:
df_final.classe.value_counts(normalize=True)


classe
0    0.893857
1    0.106143
Name: proportion, dtype: float64

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X = df_final['texto']
y = df_final['classe']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=SEED,stratify=y)
X_to_fitTFidf=X_train.sample(0.1)
# Cria o classificador MLP



MemoryError: 

In [ ]:
tfidf_vec_ssl = TfidfVectorizer(ngram_range=(1,3), strip_accents='unicode', lowercase=True, max_features=3000, min_df=10)

X_vec = tfidf_vec_ssl.fit_transform(X_train)
X_vec = X_vec.toarray()
y_vec = y_train
print(X_vec.shape)

In [ ]:
from sklearn.model_selection import GridSearchCV
# Define a grade de hiperparâmetros
param_grid = {
    # Diferentes combinações para as duas camadas ocultas:
    'hidden_layer_sizes': [(50, 50), (100, 50), (100, 100), (150, 50)],
    # Funções de ativação: tanh e logistic (sigmoid)
    'activation': ['tanh', 'logistic'],
    # Solver pode ser ajustado também, aqui usamos o 'adam' como exemplo
    'solver': ['adam']
}

# Cria o classificador MLP
mlp = MLPClassifier(max_iter=500, random_state=42)

# Configura o GridSearchCV para buscar os melhores parâmetros
grid_search = GridSearchCV(estimator=mlp,
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           verbose=2)

# Executa a busca nos dados de treinamento
grid_search.fit(X_train, y_train)

# Exibe os melhores parâmetros e a pontuação associada
print("Melhores parâmetros encontrados:")
print(grid_search.best_params_)
print("Melhor pontuação (CV): {:.4f}".format(grid_search.best_score_))

# Avalia o modelo no conjunto de teste
score = grid_search.score(X_test, y_test)
